In [92]:
import pandas as pd
import numpy as np
import re
import os
from sklearn.preprocessing import LabelEncoder

In [93]:
bayut = pd.read_csv("bayut_properties.csv")
dubizzle = pd.read_csv("dubizzle_properties.csv")

In [94]:
bayut["source"] = "bayut"
dubizzle["source"] = "dubizzle"


In [95]:
df = pd.concat([bayut, dubizzle], ignore_index=True)
print(f" Combined dataset shape: {df.shape}")

 Combined dataset shape: (6356, 15)


In [96]:
df

,bathrooms,bedrooms,furnishing,link,listing_id,listing_type,location,price,property_type,purpose,size_sqm,title,source,added_on,completion
0,5 Baths,5 Beds,Furnished,https://www.bayut.om/en/property/details-50002...,500024656,Villa,"Al Buraimi, Al Buraimi","1,200",Villa,For Sale,"1,500 Sq. M.",5 Bedrooms Villa for Sale in Oman,bayut,NaN,NaN
1,3 Baths,3 Beds,Furnished,https://www.bayut.om/en/property/details-50002...,500024649,Apartment,"Mahut, Al Wusta","1,200",Apartment,For Sale,"1,000 Sq. M.","3 Bedrooms Apartment For Sale Mahut, Al Wusta",bayut,NaN,NaN
2,5 Baths,5 Beds,Furnished,https://www.bayut.om/en/property/details-50002...,500024648,Villa,"Qantab, Muscat","1,500",Villa,For Sale,"1,500 Sq. M.",5 Bedrooms Villa For Sale in Omani City,bayut,NaN,NaN
3,5 Baths,5 Beds,Furnished,https://www.bayut.om/en/property/details-50002...,500024621,Villa,"Al Sifah, Muscat","1,500",Villa,For Sale,"1,000 Sq. M.","5 Bedrooms Villa For Sale in Al Sifah, Muscat",bayut,NaN,NaN
4,4 Baths,4 Beds,Furnished,https://www.bayut.om/en/property/details-50002...,500024573,Villa,"Hamra, Al Dakhiliya","1,000",Villa,For Sale,"1,000 Sq. M.","4 Bedrooms Villa For Sale in Hamra, Al Dakhiliya",bayut,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6351,NaN,NaN,NaN,https://www.dubizzle.com.om/en/ad/%D8%A7%D8%B1...,ID129835375.html,For Sale,"Suwaiq, Al Batinah","OMR 13,000",NaN,Sale,NaN,ارض قريب البحر للبيع,dubizzle,NaN,NaN
6352,NaN,NaN,NaN,https://www.dubizzle.com.om/en/ad/%D8%A7%D8%B3...,ID130236522.html,For Sale,"Suwaiq, Al Batinah","OMR 85,000",NaN,Sale,NaN,استراحه جديده للبيع,dubizzle,NaN,NaN
6353,NaN,NaN,NaN,https://www.dubizzle.com.om/en/ad/%D8%A3%D8%B1...,ID130223202.html,For Sale,"Ibri, Al Dhahirah","OMR 2,200",NaN,Sale,NaN,أرض الطيب موقع جميل,dubizzle,NaN,NaN
6354,NaN,NaN,NaN,https://www.dubizzle.com.om/en/ad/all-types-of...,ID129420949.html,For Sale,"Al Rusayl, Muscat","OMR 2,000",NaN,Sale,NaN,all types of interlock avaialble watsapp me 95...,dubizzle,NaN,NaN


In [97]:
import re

# Keep only English letters, numbers, and spaces
df["title"] = df["title"].astype(str).apply(lambda x: ' '.join(re.findall(r'[A-Za-z0-9]+', x)))


In [98]:
empty_titles = df[df["title"].str.strip() == ""]


In [99]:
print(f"Rows with empty title: {len(empty_titles)}")
print(empty_titles[["title", "location", "price"]].head())


Rows with empty title: 2201
     title                            location        price
1081        Al Kamil and Al Waafi, Al Sharqiya        7,500
1494                           Al Hail, Muscat          115
1802                       Nizwa, Al Dakhiliya       59,000
1810                             Ghala, Muscat   OMR 46,000
1844                       Al Maabilah, Muscat  OMR 120,000


In [122]:
print(df.isnull().sum())


bathrooms       0
bedrooms        0
link            0
listing_type    0
location        0
price           0
purpose         0
size_sqm        0
title           0
source          0
dtype: int64


In [101]:
df["size_sqm"] = (
    df["size_sqm"]
    .astype(str)
    .str.replace("Sq. M.", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)


In [102]:
df["size_sqm"] = pd.to_numeric(df["size_sqm"], errors="coerce")


In [82]:
median_size = df["size_sqm"].median()
df["size_sqm"] = df["size_sqm"].fillna(median_size)


In [103]:
df.drop(columns=["listing_id","added_on","completion","furnishing","property_type"], inplace=True)

In [104]:
df.dropna(subset=["bedrooms", "bathrooms"], inplace=True)

## price

In [105]:
df["price"] = (
    df["price"]
    .astype(str)
    .str.replace("OMR", "", regex=False)
    .str.replace(",", "", regex=False)
    .str.strip()
)


In [106]:
df["price"] = pd.to_numeric(df["price"], errors="coerce")


In [133]:
df

,bathrooms,bedrooms,link,listing_type,location,price,purpose,size_sqm,title,source,price_per_sqm,beds_bin
0,5.0,5,https://www.bayut.om/en/property/details-50002...,Villa,"Al Buraimi, Al Buraimi",1200.0,For Sale,1500.0,5 Bedrooms Villa for Sale in Oman,bayut,0.8,"(4, 6]"
1,3.0,3,https://www.bayut.om/en/property/details-50002...,Apartment,"Mahut, Al Wusta",1200.0,For Sale,1000.0,3 Bedrooms Apartment For Sale Mahut Al Wusta,bayut,1.2,"(2, 3]"
2,5.0,5,https://www.bayut.om/en/property/details-50002...,Villa,"Qantab, Muscat",1500.0,For Sale,1500.0,5 Bedrooms Villa For Sale in Omani City,bayut,1.0,"(4, 6]"
3,5.0,5,https://www.bayut.om/en/property/details-50002...,Villa,"Al Sifah, Muscat",1500.0,For Sale,1000.0,5 Bedrooms Villa For Sale in Al Sifah Muscat,bayut,1.5,"(4, 6]"
4,4.0,4,https://www.bayut.om/en/property/details-50002...,Villa,"Hamra, Al Dakhiliya",1000.0,For Sale,1000.0,4 Bedrooms Villa For Sale in Hamra Al Dakhiliya,bayut,1.0,"(3, 4]"
...,...,...,...,...,...,...,...,...,...,...,...,...
6330,4.0,4,https://www.dubizzle.com.om/en/ad/three-levels...,For Sale,"Al Khoud, Muscat",130000.0,Sale,200.0,three levels house innkhoud 3 flats rented,dubizzle,650.0,"(3, 4]"
6334,5.0,6,https://www.dubizzle.com.om/en/ad/amazing-hous...,For Sale,"Al Khoud, Muscat",125000.0,Sale,200.0,amazing house near souq al khoud,dubizzle,625.0,"(4, 6]"
6338,3.0,2,https://www.dubizzle.com.om/en/ad/2bhk-apartme...,For Sale,"Muscat Hills, Muscat",85000.0,Sale,200.0,2bhk apartment for sale,dubizzle,425.0,"(1, 2]"
6348,4.0,3,https://www.dubizzle.com.om/en/ad/%D9%81%D9%84...,For Sale,"Barka, Al Batinah",49000.0,Sale,200.0,,dubizzle,245.0,"(2, 3]"


In [134]:
df.groupby('location')['price'].mean()



location
Adam, Al Dakhiliya                39000.000000
Al Amarat, Muscat                 69703.373418
Al Buraimi, Al Buraimi           107823.076923
Al Bustan, Muscat                 51470.000000
Al Duqum, Al Wusta                39984.782609
                                     ...      
Thumrait, Dhofar                   8266.666667
Wadi Al Ma'awal, Al Batinah      100000.000000
Wadi Bani Khalid, Al Sharqiya      8000.000000
Yiti, Muscat                     238409.881818
Yunqul, Al Dhahirah               49900.000000
Name: price, Length: 74, dtype: float64

In [126]:



pat_int   = r'(\d+)'                # bedrooms
pat_float = r'(\d+(?:\.\d+)?)'      # bathrooms (keeps 3.5)


df['bedrooms'] = (pd.to_numeric(
    df['bedrooms'].astype(str).str.extract(pat_int, expand=False),
    errors='coerce'
).astype('Int64'))


df['bathrooms'] = (pd.to_numeric(
    df['bathrooms'].astype(str).str.extract(pat_float, expand=False),
    errors='coerce'
).astype('Float64'))


In [128]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3518 entries, 0 to 6349
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   bathrooms      3518 non-null   Float64 
 1   bedrooms       3518 non-null   Int64   
 2   link           3518 non-null   string  
 3   listing_type   3518 non-null   category
 4   location       3518 non-null   string  
 5   price          3518 non-null   float64 
 6   purpose        3518 non-null   category
 7   size_sqm       3518 non-null   Float64 
 8   title          3518 non-null   string  
 9   source         3518 non-null   category
 10  price_per_sqm  3518 non-null   Float64 
 11  rooms_total    3518 non-null   Float64 
 12  beds_bin       3518 non-null   category
dtypes: Float64(4), Int64(1), category(4), float64(1), string(3)
memory usage: 306.5 KB


In [119]:
df['listing_type'] = df['listing_type'].astype('category')
df['purpose']      = df['purpose'].astype('category')
df['source']       = df['source'].astype('category')

df['location']     = df['location'].astype('string')  # or 'category' if few uniques
df['title']        = df['title'].astype('string')
df['link']         = df['link'].astype('string')      # keeps URL text clean


In [111]:
df['bathrooms'] = df['bathrooms'].astype('Float64')
df['bedrooms']  = df['bedrooms'].astype('Float64')


In [113]:
df['size_sqm'] = pd.to_numeric(
    df['size_sqm'].astype(str).str.replace(',', '', regex=False)
                  .str.extract(r'(\d+(?:\.\d+)?)', expand=False),
    errors='coerce'
).astype('Float64')

df['size_sqm'].fillna(df['size_sqm'].median(), inplace=True)

C:\Users\bbuser\AppData\Local\Temp\ipykernel_5524\637820612.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['size_sqm'].fillna(df['size_sqm'].median(), inplace=True)


In [116]:
df.dropna(subset=['bedrooms'], inplace=True)


## FEatures

In [123]:
df['price_per_sqm'] = (df['price'] / df['size_sqm']).replace([np.inf, -np.inf], np.nan)



In [124]:
df['rooms_total']   = (df['bedrooms'] + df['bathrooms']).round(1)



In [125]:

df['beds_bin'] = pd.cut(df['bedrooms'], bins=[0,1,2,3,4,6,15], right=True)

In [129]:
# URL is usually unique but incase there ar dublicates drop here
df = df.drop_duplicates(subset=['link'])



In [130]:
df['rooms_total'] = (df['bedrooms'] + df['bathrooms']).round(1)



C:\Users\bbuser\AppData\Local\Temp\ipykernel_5524\1637414179.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rooms_total'] = (df['bedrooms'] + df['bathrooms']).round(1)


In [132]:
df.drop(columns=['rooms_total'], errors='ignore', inplace=True)


C:\Users\bbuser\AppData\Local\Temp\ipykernel_5524\304732328.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['rooms_total'], errors='ignore', inplace=True)


In [136]:
output_path = os.path.join("..", "data", "processed", "cleaned_properties.csv")
os.makedirs(os.path.dirname(output_path), exist_ok=True)
df.to_csv(output_path, index=False)

print(f"\n Cleaned data saved to: {output_path}")
print(f" Final shape: {df.shape}")


 Cleaned data saved to: ..\data\processed\cleaned_properties.csv
 Final shape: (3251, 12)


In [137]:
output_path

'..\\data\\processed\\cleaned_properties.csv'